In [3]:
import plotly as ply
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import numpy as np

from haversine import haversine

# Load Camera/Fire Dataset & Count
Load Camera and Fire dataset and find all fires which more than 1 camera saw

In [4]:
df = pd.read_csv("Camera_Fire_Distances2.csv")
df['type'] = ['fire' for i in range(len(df))]

In [5]:
df.head()

,name,cam_lat,cam_long,elev,code,dir,fov_center,fov_lft,fov_rt,num_events,event_dates,x_pixel,y_pixel,incident,incident_dist,incident_latitude,incident_longitude,check,type
0,Black Mountain,32.981668,-117.116407,1600,bl,s,"[-117.12, 32.52914378468391]","[-116.74311609844423, 32.660630738922485]","[-117.49688390155578, 32.660630738922485]",4,20180603,1229,685,Adobe Fire,14.195302,32.78031,-117.067820,1,fire
1,Red Mountain,33.400800,-117.190500,1600,rm,e,"[-116.65303446254079, 33.4008]","[-116.80905606186775, 33.71897645296838]","[-116.81183156739415, 33.081443131239055]",5,20180522,1329,780,Agua Fire,9.711898,33.35505,-117.031340,1,fire
2,High Point,33.363018,-116.836220,6180,hp,n,"[-116.84, 33.81079597072698]","[-117.22126369140182, 33.6781794593324]","[-116.4587363085982, 33.678179459332405]",9,20180704,1552,761,Benton Fire,15.097142,33.56349,-116.732032,1,fire
3,Big Black Mountain,33.159900,-116.808100,4055,bm,n,"[-116.8081, 33.61071054911843]","[-117.18848493961107, 33.478094173241445]","[-116.42771506038892, 33.478094173241445]",3,20170625,1613,784,Black Fire,2.088121,33.16830,-116.773420,1,fire
4,Mesa Grande,33.188869,-116.760651,4280,mg,s,"[-116.761, 32.73715879567527]","[-116.38323866782441, 32.86863685756763]","[-117.13876133217558, 32.86863685756762]",4,20170625,1429,629,Black Fire,1.601569,33.16830,-116.773420,1,fire


In [6]:
df = df.loc[df['check'] == 1]
fire_counts_filtered = df.groupby(['incident', 'event_dates'])['check'].sum()
fire_counts_filtered = fire_counts_filtered[fire_counts_filtered > 1]
fire_counts_filtered

incident         event_dates
Black Fire       20170625       2
Border 11 Fire   20200727       2
Border 9 Fire    20190814       2
Border Fire      20160619       3
Bridle Fire      20180614       2
Casner Fire      20180806       2
Cloverdale Fire  20180813       3
Dehesa Fire      20190728       2
Dehesa Fire      20190924       5
Feather Fire     20160718       3
Gate Fire        20170520       3
Gopher Fire      20161113       3
Jennings Fire    20170711       5
Lilac Fire       20171207       2
Lost Fire        20170722       4
Meadow Fire      20190716       3
Pasqual Fire     20180727       4
Pauma Fire       20190610       2
Portola Fire     20171010       3
Ranch 2 Fire     20200813       4
Rangeland Fire   20180809       3
Recycle Fire     20180606       3
Rock Fire        20160722       2
Valley Fire      20200905       3
Vista Fire       20190717       2
West Fire        20190813       2
Willow Fire      20190924       4
                 20190925       2
                 20

In [7]:
landmark_df = pd.read_csv("landmarks_processed.csv")
landmark_df['type'] = ['landmark' for i in range(len(landmark_df))]

for fire in fire_counts_filtered.keys():
    fire_name = fire[0]
    date = fire[1]
    code_list = df[['code','dir']].loc[(df['incident'] == fire_name) & (df['event_dates'] == date)].reset_index(drop=True)
    fires_len = len(df.merge(code_list, left_on = ['code','dir'], right_on=['code', 'dir'], how="inner"))
    merge_len = len(landmark_df.merge(code_list, left_on = ['station','dir'], right_on=['code', 'dir'], how="inner"))
    print("{}\t Fires: {}\t Landmark: {}".format(fire_name, fires_len, merge_len))

Black Fire	 Fires: 7	 Landmark: 4
Border 11 Fire	 Fires: 6	 Landmark: 2
Border 9 Fire	 Fires: 6	 Landmark: 2
Border Fire	 Fires: 6	 Landmark: 3
Bridle Fire	 Fires: 17	 Landmark: 3
Casner Fire	 Fires: 5	 Landmark: 3
Cloverdale Fire	 Fires: 8	 Landmark: 4
Dehesa Fire	 Fires: 8	 Landmark: 4
Dehesa Fire 	 Fires: 16	 Landmark: 5
Feather Fire	 Fires: 19	 Landmark: 6
Gate Fire	 Fires: 9	 Landmark: 3
Gopher Fire	 Fires: 6	 Landmark: 3
Jennings Fire	 Fires: 13	 Landmark: 6
Lilac Fire	 Fires: 4	 Landmark: 3
Lost Fire	 Fires: 7	 Landmark: 4
Meadow Fire	 Fires: 12	 Landmark: 3
Pasqual Fire	 Fires: 10	 Landmark: 7
Pauma Fire	 Fires: 6	 Landmark: 2
Portola Fire	 Fires: 11	 Landmark: 4
Ranch 2 Fire	 Fires: 6	 Landmark: 4
Rangeland Fire	 Fires: 9	 Landmark: 4
Recycle Fire	 Fires: 5	 Landmark: 2
Rock Fire	 Fires: 13	 Landmark: 5
Valley Fire	 Fires: 12	 Landmark: 3
Vista Fire	 Fires: 10	 Landmark: 2
West Fire 	 Fires: 2	 Landmark: 6
Willow Fire	 Fires: 16	 Landmark: 5
Willow Fire	 Fires: 11	 Landmark: 2

In [8]:
df

,name,cam_lat,cam_long,elev,code,dir,fov_center,fov_lft,fov_rt,num_events,event_dates,x_pixel,y_pixel,incident,incident_dist,incident_latitude,incident_longitude,check,type
0,Black Mountain,32.981668,-117.116407,1600,bl,s,"[-117.12, 32.52914378468391]","[-116.74311609844423, 32.660630738922485]","[-117.49688390155578, 32.660630738922485]",4,20180603,1229,685,Adobe Fire,14.195302,32.780310,-117.067820,1,fire
1,Red Mountain,33.400800,-117.190500,1600,rm,e,"[-116.65303446254079, 33.4008]","[-116.80905606186775, 33.71897645296838]","[-116.81183156739415, 33.081443131239055]",5,20180522,1329,780,Agua Fire,9.711898,33.355050,-117.031340,1,fire
2,High Point,33.363018,-116.836220,6180,hp,n,"[-116.84, 33.81079597072698]","[-117.22126369140182, 33.6781794593324]","[-116.4587363085982, 33.678179459332405]",9,20180704,1552,761,Benton Fire,15.097142,33.563490,-116.732032,1,fire
3,Big Black Mountain,33.159900,-116.808100,4055,bm,n,"[-116.8081, 33.61071054911843]","[-117.18848493961107, 33.478094173241445]","[-116.42771506038892, 33.478094173241445]",3,20170625,1613,784,Black Fire,2.088121,33.168300,-116.773420,1,fire
4,Mesa Grande,33.188869,-116.760651,4280,mg,s,"[-116.761, 32.73715879567527]","[-116.38323866782441, 32.86863685756763]","[-117.13876133217558, 32.86863685756762]",4,20170625,1429,629,Black Fire,1.601569,33.168300,-116.773420,1,fire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Mt. Woodson,33.008700,-116.970900,2800,wc,s,"[-116.9709, 32.55784585299181]","[-116.59389556804973, 32.68933158235236]","[-117.34790443195027, 32.68933158235236]",2,20190925,83,1014,Willow Fire,10.436240,32.878333,-116.880000,1,fire
252,Otay Mountain,32.594800,-116.844700,3565,om,n,"[-116.8447, 33.04565148266326]","[-117.22264979421115, 32.9130354508156]","[-116.46675020578886, 32.913035450815606]",10,20201202,1030,1101,Willow Fire,11.510103,32.752778,-116.907500,1,fire
253,Santiago Peak,33.711172,-117.534115,5700,stgo,e,"[0, 0]","[0, 0]","[0, 0]",3,20180827,1025,1189,Winchester Fire,26.266977,33.640051,-117.085354,1,fire
254,High Point,33.363018,-116.836220,6180,hp,n,"[-116.84, 33.81079597072698]","[-117.22126369140182, 33.6781794593324]","[-116.4587363085982, 33.678179459332405]",9,20190629,561,775,Wolf Fire,38.761284,33.911000,-116.980528,1,fire


In [14]:
fire_counts_filtered.keys()[1][1]

20200727

In [32]:
fire_name = fire_counts_filtered.keys()[15][0]
print(fire_name)
fire_date = fire_counts_filtered.keys()[15][1]
print(fire_date)
code_list = df[['code','dir']].loc[(df['incident'] == fire_name) & (df['event_dates'] == fire_date)].reset_index(drop=True)


cols = ['station', 'dir', 'lat', 'long', 'x_pixel', 'y_pixel', 'distance', 'landmark', 'type']
df_plot = pd.DataFrame(columns=cols)
filt_df = df.merge(code_list, left_on = ['code','dir'], right_on=['code', 'dir'], how="inner").rename(columns={'incident': 'landmark',
                                                                                            'code':'station',
                                                                                            'incident_latitude': 'lat',
                                                                                            'incident_longitude': 'long',
                                                                                            'incident_dist': 'distance',
                                                                                            'incident': 'landmark',
                                                                                            'event_dates': 'date'})
df_plot = pd.concat([df_plot, filt_df[cols], landmark_df[cols].merge(code_list, left_on = ['station','dir'], right_on=['code', 'dir'], how="inner")])
df_plot = pd.concat([df_plot, df.loc[(df['incident'] == fire_name) & (df['event_dates'] == fire_date)].reset_index(drop=True).rename(columns={'cam_lat': 'lat', 'cam_long': 'long', 'code':'station'})[['lat', 'long', 'station', 'dir']]])
df_plot['code'] = df_plot['station']
df_plot['type'] = df_plot['type'].fillna('station')
df_plot['landmark'] = df_plot['landmark'].fillna(df_plot['code'] + " Station")
df_plot = df_plot.reset_index(drop=True)

Meadow Fire
20190716


In [33]:
df_plot

,station,dir,lat,long,x_pixel,y_pixel,distance,landmark,type,code
0,hp,n,33.563490,-116.732032,1552,761,15.097142,Benton Fire,fire,hp
1,hp,n,33.524412,-116.821724,1062,792,11.182537,Bridle Fire,fire,hp
2,hp,n,33.514658,-116.788400,1265,784,10.834029,Meadow Fire,fire,hp
3,hp,n,33.504880,-117.021320,42,852,14.490812,Portola Fire,fire,hp
4,hp,n,33.525840,-116.852260,434,769,11.287872,Reed Fire,fire,hp
5,hp,n,33.571763,-116.500058,780,741,24.154362,Ribbon Fire,fire,hp
6,hp,n,33.551838,-116.746417,1358,767,14.035721,Tripp Fire,fire,hp
7,hp,n,33.911000,-116.980528,561,775,38.761284,Wolf Fire,fire,hp
8,mg,n,33.346590,-116.661070,1438,957,12.322739,Lost Fire,fire,mg
9,mg,n,33.514658,-116.788400,1564,951,22.566776,Meadow Fire,fire,mg


In [17]:
def find_dist(x1, x2, y1, y2):
    return np.sqrt((y2 - y1)**2 + (x2-x1)**2)

def find_angle(a1, a2, b1, b2, rad):
    cont = np.sqrt(((a2-a1)**2) + ((b2-b1)**2))/(2*rad)
    #print(cont)
    return 2 * np.arcsin(cont)
    #return np.arctan2( b2 - b1, a2 - a1 )

def find_fov_old(cx, cy, px, py, d, pix):
    R = find_dist(cx, px, cy, py)
    print("Dist: ", R)
    if (d == "n") | (d == "s"):
        priority = 1
    else:
        priority = 0
    
    if d == "n":
        fx = cx
        fy = cy + R
        #ang_ref = find_angle(cx, px, cy, fy, R)
    if d == "s":
        fx = cx
        fy = cy - R
        #ang_ref = find_angle(cx, px, cy, fy, R)
    if d == "e":
        fy = cy
        fx = cx + R
        #ang_ref = find_angle(cx, fx, cy, py, R)
    if d == "w":
        fy = cy
        fx = cx - R
        #ang_ref = find_angle(cx, fx, cy, py, R)
    
    fx2 = fx# - cx
    fy2 = fy# - cy
    px2 = px# - cx
    py2 = py# - cy
    print("{}, {}, {}, {}, {}".format(fx2, px2, fy2, py2, R))
    ang_ref = find_angle(fx2, px2, fy2, py2, R)
    print("Angle: ", ang_ref)
    arclen = R*ang_ref
    print("Arc Length: ", arclen)
    ang_ref2 = (arclen*(1024/abs(1024-pix))) / R
    print("Extended Arc Length: ", R*ang_ref2)
    return ang_ref, ang_ref2
    
def find_fov_new(cx, cy, px, py, d, pix):
    R = find_dist(cx, px, cy, py)
    print("Dist: ", R)
    if (d == "n") | (d == "s"):
        priority = 1
    else:
        priority = 0
    
    if d == "n":
        fx = cx
        fy = cy + R
        #ang_ref = find_angle(cx, px, cy, fy, R)
    if d == "s":
        fx = cx
        fy = cy - R
        #ang_ref = find_angle(cx, px, cy, fy, R)
    if d == "e":
        fy = cy
        fx = cx + R
        #ang_ref = find_angle(cx, fx, cy, py, R)
    if d == "w":
        fy = cy
        fx = cx - R
        #ang_ref = find_angle(cx, fx, cy, py, R)
    
    fx2 = fx# - cx
    fy2 = fy# - cy
    px2 = px# - cx
    py2 = py# - cy
    print("{}, {}, {}, {}, {}".format(fx2, px2, fy2, py2, R))
    ang_ref = find_angle(fx2, px2, fy2, py2, R)
    print("Angle: ", ang_ref)
    arclen = R*ang_ref/2
    print("Arc Length: ", arclen)
    ang_ref2 = (arclen*(1536/abs(1536-pix))) / R
    print("Extended Arc Length: ", R*ang_ref2)
    return ang_ref, ang_ref2
    

In [18]:
df_plot

,station,dir,lat,long,x_pixel,y_pixel,distance,landmark,type,code
0,hp,n,33.563490,-116.732032,1552,761,15.097142,Benton Fire,fire,hp
1,hp,n,33.524412,-116.821724,1062,792,11.182537,Bridle Fire,fire,hp
2,hp,n,33.514658,-116.788400,1265,784,10.834029,Meadow Fire,fire,hp
3,hp,n,33.504880,-117.021320,42,852,14.490812,Portola Fire,fire,hp
4,hp,n,33.525840,-116.852260,434,769,11.287872,Reed Fire,fire,hp
5,hp,n,33.571763,-116.500058,780,741,24.154362,Ribbon Fire,fire,hp
6,hp,n,33.551838,-116.746417,1358,767,14.035721,Tripp Fire,fire,hp
7,hp,n,33.911000,-116.980528,561,775,38.761284,Wolf Fire,fire,hp
8,mg,n,33.346590,-116.661070,1438,957,12.322739,Lost Fire,fire,mg
9,mg,n,33.514658,-116.788400,1564,951,22.566776,Meadow Fire,fire,mg


In [563]:
#Find FoV Center Reference - hp
center_x1, center_y1 = df_plot.iloc[-3]['long'], df_plot.iloc[-3]['lat']
point_x1, point_y1 = -116.852260, 33.525840
pix1 = 434
direction = "n"

ang_a, angle_a = find_fov_old(center_x1, center_y1, point_x1, point_y1, direction, pix)

Dist:  0.16361016253277771
-116.83622, -116.85226, 33.526628162532774, 33.52584, 0.16361016253277771
Angle:  0.09819565050347874
Arc Length:  0.016065806338885994
Extended Arc Length:  0.8658624047904872


In [553]:
#Find FoV Center Reference - mg
center_x2, center_y2 = df_plot.iloc[-2]['long'], df_plot.iloc[-2]['lat']
point_x2, point_y2 = -116.661070, 33.346590
pix2 = 1438
direction = "n"

ang_b, angle_b = find_fov_new(center_x2, center_y2, point_x2, point_y2, direction, pix)

Dist:  0.18652691334496696
-116.760651, -116.66107, 33.37539591334497, 33.34659, 0.18652691334496696
Angle:  0.5631700190994652
Arc Length:  0.052523182675524666
Extended Arc Length:  0.16364220809250687


In [40]:
fig = px.scatter(df_plot,
                 x="long", 
                 y="lat", 
                 color="type", 
                 title="Fires Seen",
                text = "landmark",
                width=600, height=600
                )
fig.update_traces(textposition='top center')
fig.update_traces(marker=dict(size=20,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'),
                 textposition='top right')
fig.show()

In [585]:
#a FoV Left
theta_a1 = angle_a-(np.pi/2)
b1 = center_y1 - (np.tan(theta_a1) * center_x1)
x1 = np.linspace(center_x1-.2, center_x1)
y1 = x1*np.tan(theta_a1) + b1

#a FoV Right
theta_a2 = -theta_a1# - np.pi
b2 = center_y1 - (np.tan(theta_a2) * center_x1)
x2 = np.linspace(center_x1, center_x1+.2)
y2 = x2*np.tan(theta_a2) + b2

#Center Line
theta_b1 = -((np.pi/2)-angle_b)
b3 = center_y1 - (np.tan(theta_b1) * center_x1)
x3 = np.linspace(-117,-116.6)
y3 = x3*np.tan(theta_b1) + b3

#Center Line
#theta_b2 = -theta_b1 + np.pi
#b4 = center_y2 - (np.tan(theta_b2) * center_x2)
#x4 = np.linspace(-116.56, -116.96)
#y4 = x4*np.tan(theta_b2) + b4


#mg FoV Left
theta_mg1 = angle_mg-(np.pi/2)
b5 = center_y2 - (np.tan(theta_mg1) * center_x2)
x5 = np.linspace(center_x2-.2, center_x2)
y5 = x5*np.tan(theta_mg1) + b5

#mg FoV Right
theta_mg2 = -theta_mg1 - np.pi
b6 = center_y2 - (np.tan(theta_mg2) * center_x2)
x6 = np.linspace(center_x2, center_x2+.2)
y6 = x6*np.tan(theta_mg2) + b6
"""

#Guess MG
#ratio = (abs(1536-1030))/1536
#b7 = center_y2 - (ratio * center_x2)
#x7 = np.linspace(-117.261, -116.761)
#y7 = x7*ratio + b7

ang_mg_g = (angle_mg*(1-((1536-1030))/1536))# + np.pi/2 #*(1536-1040/1536)) #(abs(3072-1040)/3072)
b7 = center_y2 - (np.tan(ang_mg_g) * center_x2)
x7 = np.linspace(-117.061, -116.761)
y7 = x7*np.tan(ang_mg_g) + b7
"""

#Guess 1
ratio = (1024/(1265-1024))
b8 = center_y1 - (ratio * center_x1)
x8 = np.linspace(center_x1, center_x1+.2)
y8 = x8*ratio + b8

#ang_b_g =  (np.pi/2) - (angle_b*((1820-1024)/1024))
#ang_b_g =  (np.pi/2) - (angle_b*((1820-1024)/1024))
#b8 = center_y3 - (np.tan(ang_b_g) * center_x3)
#x8 = np.linspace(-117.12, -116.82)
#y8 = x8*np.tan(ang_b_g) + b8

#Guess 2
ratio = (1536/(1564-1536))
b9 = center_y2 - (ratio * center_x2)
x9 = np.linspace(center_x2-.2, center_x2+.2)
y9 = x9*ratio + b9




#Estimation Line a
#ratio = (1024-1520)/2048
#b3 = center_y1 - (ratio * center_x1)
#x3 = np.linspace(-117.35,-117.19050)
#y3 = x3*ratio + b3

In [536]:
import numpy as np

a = np.array([-117.0057, 33.05488])
b = np.array([-117.1164071, 32.98166822])
c = np.array([-117.2343, 33.05488])

ba = a - b
bc = c - b

cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
angle = np.arccos(cosine_angle)

print(np.degrees(angle))

114.68253051451445


In [429]:
x_int = -(b8 - b7)/(np.tan(ang_bl_g) - np.tan(ang_mg_g))
y_int = x_int*np.tan(ang_bl_g) + b8

In [430]:
print("Point of Intersection: {}, {}".format(x_int, y_int) )

Point of Intersection: 68337.01507888117, 76794.33659330643


In [431]:
dist = haversine((x_int, y_int), (-117.0242529, 33.0971179))
print("Distance is {}km or {}mi".format(dist, 0.621371*dist))

Distance is 4485.956002645202km or 2787.442967319652mi


In [576]:
df_plot

,station,dir,lat,long,x_pixel,y_pixel,distance,landmark,type,code
0,hp,n,33.525840,-116.852260,434,769,11.287872,Reed Fire,fire,hp
1,hp,n,33.504880,-117.021320,42,852,14.490812,Portola Fire,fire,hp
2,hp,n,33.524412,-116.821724,1062,792,11.182537,Bridle Fire,fire,hp
3,hp,n,33.563490,-116.732032,1552,761,15.097142,Benton Fire,fire,hp
4,hp,n,33.571763,-116.500058,780,741,24.154362,Ribbon Fire,fire,hp
5,hp,n,33.911000,-116.980528,561,775,38.761284,Wolf Fire,fire,hp
6,hp,n,33.514658,-116.788400,1265,784,10.834029,Meadow Fire,fire,hp
7,hp,n,33.551838,-116.746417,1358,767,14.035721,Tripp Fire,fire,hp
8,mg,n,33.346590,-116.661070,1438,957,12.322739,Lost Fire,fire,mg
9,mg,n,33.514658,-116.788400,1564,951,22.566776,Meadow Fire,fire,mg


In [586]:
fig = px.scatter(df_plot,
                 x="long", 
                 y="lat", 
                 color="type", 
                 title="Station Camera & Fires Seen",
                text = "landmark",
                width=1000, height=1000)
fig.update_traces(textposition='top center')
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'),
                 textposition='top right')

#BH
fig.add_trace(go.Scatter(x=x1, y=y1,
                    mode='lines',
                    name='left fov 1',
                     textfont=dict(size=24),
                       marker = {'color' : 'lightcoral'}))

fig.add_trace(go.Scatter(x=x2, y=y2,
                    mode='lines',
                    name='right fov 1',
                     textfont=dict(size=24),
                        marker = {'color' : 'lightcoral'}))

fig.add_trace(go.Scatter(x=x9, y=y9,
                    mode='lines',
                    name='Guess 2',
                     textfont=dict(size=24),
                        marker = {'color' : 'lightsteelblue'}))

#BL
fig.add_trace(go.Scatter(x=x5, y=y5,
                    mode='lines',
                         name='left fov 2',
                     textfont=dict(size=24),
                        marker = {'color' : 'lightsteelblue'}))

fig.add_trace(go.Scatter(x=x6, y=y6,
                    mode='lines',
                         name='right fov 2',
                        marker = {'color' : 'lightsteelblue'}))

fig.add_trace(go.Scatter(x=x8, y=y8,
                    mode='lines',
                    name='Guess 1',
                        marker = {'color' : 'lightcoral'}))



fig.show()

In [541]:
df_plot

,station,dir,lat,long,x_pixel,y_pixel,distance,landmark,type,code
0,hp,n,33.525840,-116.852260,434,769,11.287872,Reed Fire,fire,hp
1,hp,n,33.504880,-117.021320,42,852,14.490812,Portola Fire,fire,hp
2,hp,n,33.524412,-116.821724,1062,792,11.182537,Bridle Fire,fire,hp
3,hp,n,33.563490,-116.732032,1552,761,15.097142,Benton Fire,fire,hp
4,hp,n,33.571763,-116.500058,780,741,24.154362,Ribbon Fire,fire,hp
5,hp,n,33.911000,-116.980528,561,775,38.761284,Wolf Fire,fire,hp
6,hp,n,33.514658,-116.788400,1265,784,10.834029,Meadow Fire,fire,hp
7,hp,n,33.551838,-116.746417,1358,767,14.035721,Tripp Fire,fire,hp
8,mg,n,33.346590,-116.661070,1438,957,12.322739,Lost Fire,fire,mg
9,mg,n,33.514658,-116.788400,1564,951,22.566776,Meadow Fire,fire,mg


In [22]:
#Todo
#See if we get the same field of view when using other fire events than vista